<a href="https://colab.research.google.com/github/Patryk-O/Konwulcyjne-sieci-neuronowe/blob/main/projectsieci_transferowa_z_odmro%C5%BCeniem_warstw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
from google.colab import drive
from matplotlib import pyplot as plt
from keras.utils import image_dataset_from_directory
from keras import layers, Model, Input
from keras.models import Sequential
from keras.utils import plot_model
from keras.metrics import CategoricalAccuracy, TruePositives, TrueNegatives, FalsePositives, FalseNegatives,Accuracy
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from keras.applications import Xception, xception

In [ ]:
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/grzyby/grzyby.zip"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
replace grzyby/test/H1/H1_102a_19.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:

directory_test = '/content/grzyby/test/'
directory_train = '/content/grzyby/train/'

H1_train = os.path.join(directory_train, 'H1')
H2_train = os.path.join(directory_train, 'H2')
H3_train = os.path.join(directory_train, 'H3')
H5_train = os.path.join(directory_train, 'H5')
H6_train = os.path.join(directory_train, 'H6')


H1_test = os.path.join(directory_test, 'H1')
H2_test = os.path.join(directory_test, 'H2')
H3_test = os.path.join(directory_test, 'H3')
H5_test = os.path.join(directory_test, 'H5')
H6_test = os.path.join(directory_test, 'H6')


print('Zbiór uczący')
print(f'Liczba zdjęć H1 {len(os.listdir(H1_train))}')
print(f'Liczba zdjęć H2 {len(os.listdir(H2_train))}')
print(f'Liczba zdjęć H3 {len(os.listdir(H3_train))}')
print(f'Liczba zdjęć H5 {len(os.listdir(H5_train))}')
print(f'Liczba zdjęć H6 {len(os.listdir(H6_train))}')


print('Zbiór testowy')
print(f'Liczba zdjęć H1 {len(os.listdir(H1_test))}')
print(f'Liczba zdjęć H2 {len(os.listdir(H2_test))}')
print(f'Liczba zdjęć H3 {len(os.listdir(H3_test))}')
print(f'Liczba zdjęć H5 {len(os.listdir(H5_test))}')
print(f'Liczba zdjęć H6 {len(os.listdir(H6_test))}')




In [ ]:
train_dataset, valid_dataset = image_dataset_from_directory(directory_train, validation_split=0.2,
                                       subset='both',
                                       seed=1410,
                                       image_size=(256, 256),
                                       label_mode='categorical',
                                      color_mode="rgb"
                                       )

test_dataset = image_dataset_from_directory(directory_test, seed=1410,
                                       image_size=(256, 256),
                                       label_mode='categorical',
                                      color_mode="rgb"
                                       )


In [ ]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
]


def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

In [ ]:
class_names = train_dataset.class_names
print(class_names)

%matplotlib inline
fig, ax = plt.subplots(4,4, figsize=(10,10))


for images, labels in train_dataset.take(1):
    for i in range(16):
        augmented_images = data_augmentation(images)

        ax = plt.subplot(4,4,i + 1)
        plt.imshow(augmented_images[i].numpy().astype("uint8"))
        plt.title(class_names[int(np.argmax(labels[i]))])
        plt.axis("off")


In [ ]:
train_dataset = train_dataset.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf_data.AUTOTUNE,
)

# Prefetching samples in GPU memory helps maximize GPU utilization.
train_dataset = train_dataset.prefetch(tf_data.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(tf_data.AUTOTUNE)

In [ ]:
base_model = Xception(
    weights='imagenet',
    input_shape=(256, 256, 3),
    include_top=False)

base_model.trainable = False

plot_model(base_model, show_shapes=True)

In [ ]:
#warstwy wejściowe
inputs = Input(shape=(256, 256, 3))
x = xception.preprocess_input(inputs)
x = base_model(x, training=False)
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(5, activation='softmax')(x)

model = Model(inputs ,x)

In [ ]:
# Kompilacja
model.compile(optimizer='adam', loss="categorical_crossentropy", metrics=['accuracy'])

# Uczenie sieciSS
history = model.fit(train_dataset, epochs=30, validation_data=valid_dataset,
                    batch_size=32)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.0, 1])
plt.legend(loc='lower right')

In [ ]:
result = model.evaluate(test_dataset, return_dict=True)

print(f'Dokładność modelu {round(result["accuracy"]*100,2)}%')

In [ ]:
y_pred = model.predict(test_dataset)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.concatenate([y for x, y in test_dataset], axis=0)
y_true = np.argmax(y_true, axis=1)  # Przekształcenie na etykiety klas.

In [ ]:
from sklearn.metrics import confusion_matrix, balanced_accuracy_score, f1_score

# Załóżmy, że y_true to prawdziwe etykiety, a y_pred to przewidziane etykiety
confusion_matrix = confusion_matrix(y_true, y_pred)
print(confusion_matrix)
bac = balanced_accuracy_score(y_true, y_pred)
print(bac)
f1_macro = f1_score(y_true, y_pred, average='macro')
f1_weighted = f1_score(y_true, y_pred, average='weighted')
f1_micro = f1_score(y_true, y_pred, average='micro')

print(f'F1 Score (Macro): {f1_macro}')
print(f'F1 Score (Weighted): {f1_weighted}')
print(f'F1 Score (Micro): {f1_micro}')

